In [1]:
from transformers import AutoTokenizer, BartForConditionalGeneration
save_path = f"/home/jovyan/hdfs-jmt-rungjoo-private/save_models/facet/baseline_bart"
# model_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/bart-base"

model = BartForConditionalGeneration.from_pretrained(save_path)
model = model.cuda()
model.eval()

tokenizer = AutoTokenizer.from_pretrained(save_path)
print("ok")

/home/jovyan/venv_p5/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ok


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [12]:
query_string = "100 best weight watchers recipes"
query_string = "1016"
query_string = "caesars atlantic city"
inputs = tokenizer(query_string, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors='pt', add_special_tokens=False)
inputs.to(device)

{'input_ids': tensor([[ 3245,   293,  2726,    23,   462, 26970,   343]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [13]:
tokenizer.decode(inputs['input_ids'][0])

'caesars atlantic city'

In [21]:
token_ids = model.generate(inputs["input_ids"])
pred_facet_string = tokenizer.decode(token_ids[0], skip_special_tokens=True)

In [24]:
[x.strip() for x in pred_facet_string.split(",")]

['caesars parking atlantic city', 'caesars events atlatic city']

In [26]:
test_result = {1:2}
with open("result/baseline.json", "w", encoding='utf-8') as f:
    json.dump(test_result, f)

## 비교모델

### gpt3

In [52]:
import jsonlines

gpt3_facets = {}
with jsonlines.open("Faspect/gpt3_facets.jsonl") as f:
    for line in f.iter():
        query = line['query']
        pred = line['facets']
        gpt3_facets[query] = pred

In [51]:
import json
with open("result/baseline.json", 'r', encoding='utf-8') as f:
    baseline = json.load(f)

In [54]:
gpt3_facets_result = {}
for ind, data in baseline.items():
    query = data['query']
    label = data['label']
    pred = gpt3_facets[query]
    
    gpt3_facets_result[ind] = {}
    gpt3_facets_result[ind]['query'] = query
    gpt3_facets_result[ind]['pred'] = pred
    gpt3_facets_result[ind]['label'] = label

In [55]:
with open("result/gpt3_facets.json", "w", encoding='utf-8') as f:
    json.dump(gpt3_facets_result, f)

## evaluation

In [45]:
# exact_p_list, exact_r_list, exact_f1_list = [], [], []
# term_p_list, term_r_list, term_f1_list = [], [], []
# for k, data in result.items():
#     pred_list = data['pred']
#     label_list = data['label']
    
#     exact_p, exact_r, exact_f1 = exact_match(pred_list, label_list)
#     exact_p_list.append(exact_p)
#     exact_r_list.append(exact_r)
#     exact_f1_list.append(exact_f1)
    
#     term_p, term_r, term_f1 = term_overlap(pred_list, label_list)
#     term_p_list.append(term_p)
#     term_r_list.append(term_r)
#     term_f1_list.append(term_f1)
    
# exact_p_score = sum(exact_p_list)/len(exact_p_list)
# exact_r_score = sum(exact_r_list)/len(exact_r_list)
# exact_f1_score = sum(exact_f1_list)/len(exact_f1_list)

# term_p_score = sum(term_p_list)/len(term_p_list)
# term_r_score = sum(term_r_list)/len(term_r_list)
# term_f1_score = sum(term_f1_list)/len(term_f1_list)

In [46]:
# exact_p_score, exact_r_score, exact_f1_score
# term_p_score, term_r_score, term_f1_score

(0.23996873242106267, 0.23640788746720898, 0.23076348255784537)

In [4]:
# def exact_match_rung(pred_list, label_list):
#     pred_list = [x.replace(" ", "") for x in pred_list]
#     label_list = [x.replace(" ", "") for x in label_list]
    
#     pnum = 0
#     for pred in pred_list:
#         if pred in label_list:
#             pnum += 1
#     if len(pred_list) == 0:
#         precision = 0                
#     else:
#         precision = pnum/len(pred_list)
    
#     rnum = 0
#     for label in label_list:
#         if label in pred_list:
#             rnum += 1
#     if len(label_list) == 0:
#         recall = 0
#     else:
#         recall = rnum/len(label_list)
        
#     if precision == 0 or recall == 0:
#         f1 = 0
#     else:
#         f1 = 2*precision*recall/(precision+recall)
    
#     return precision, recall, f1

In [5]:
# def term_overlap_rung(pred_list, label_list):
#     pred_term = []
#     for pred in pred_list:
#         for term in pred.split(" "):
#             if term not in pred_term:
#                 pred_term.append(term)
                
#     label_term = []
#     for label in label_list:
#         for term in label.split(" "):
#             if term not in label_term:
#                 label_term.append(term)
#     pnum = 0
#     for pred in pred_term:
#         if pred in label_term:
#             pnum += 1
#     if len(pred_term) == 0:
#         precision = 0                
#     else:
#         precision = pnum/len(pred_term)
    
#     rnum = 0
#     for label in label_term:
#         if label in pred_term:
#             rnum += 1
#     if len(label_term) == 0:
#         recall = 0
#     else:
#         recall = rnum/len(label_term)
        
#     if precision == 0 or recall == 0:
#         f1 = 0
#     else:
#         f1 = 2*precision*recall/(precision+recall)
    
#     return precision, recall, f1

In [1]:
import json
with open("result/baseline.json", 'r', encoding='utf-8') as f:
    result = json.load(f)

In [9]:
import itertools

from bert_score import score
from nltk.translate.bleu_score import sentence_bleu


def best_bleu_cand(groundtruth, candidate):
#     assert len(groundtruth) >= len(candidate)
    all_permutations = list(itertools.permutations(candidate))
    max_bleu = 0.
    best_cand = all_permutations[0]
    for cand in all_permutations:
        bleu = 0.
        for i in range(min(len(groundtruth), len(cand))):
            bleu += sentence_bleu([groundtruth[i]], cand[i]) / len(groundtruth)
        if bleu > max_bleu:
            max_bleu = bleu
            best_cand = cand
    return list(best_cand)


def eval_bleu(groundtruth, cand):
    # Calculates the SET BLEU metrics, for 1-gram, 2-gram, 3-gram and 4-gram overlaps
    best_cand = best_bleu_cand(groundtruth, cand)
    bleu = [0., 0., 0., 0.]
    bleu_weights = [[1, 0, 0, 0], [0.5, 0.5, 0, 0], [0.33, 0.33, 0.33, 0], [0.25, 0.25, 0.25, 0.25]]
    for j in range(4):
        for i in range(min(len(groundtruth), len(best_cand))):
            bleu[j] += sentence_bleu([groundtruth[i]], best_cand[i], weights=bleu_weights[j]) / len(groundtruth)
    return bleu


def bertscore(groundtruth, cand):
    # Calculates the Set BERT-Score metrics for Precision, Recall & F1
    best_cand = best_bleu_cand(groundtruth, cand)
    (P, R, F), hashname = score(best_cand, groundtruth, lang="en", return_hash=True, device="cuda:0")
    return P.mean().item(), R.mean().item(), F.mean().item()


def exact_match(groundtruth, cand):
    # Calculates the exact match Precision, Recall & F1
    c = 0.
    for x in cand:
        if x != '' and x in groundtruth:
            c += 1
    p = c / (len([x for x in cand if x != ''])+1e-8)
    r = c / (len([x for x in groundtruth if x != ''])+1e-8)
    f1 = 2 * p * r / (p + r) if p + r > 0 else 0.
    return [p, r, f1]


def term_match(groundtruth, cand):
    # Calculates the term overlap Precision, Recall & F1
    gt_terms = set([])
    for x in groundtruth:
        if x == '':
            continue
        for t in x.strip().split():
            gt_terms.add(t)
    cand_terms = set([])
    for x in cand:
        if x == '':
            continue
        for t in x.strip().split():
            cand_terms.add(t)

    c = 0.
    for x in cand_terms:
        if x != '' and x in gt_terms:
            c += 1
    p = c / (len([x for x in cand_terms if x != ''])+1e-8)
    r = c / (len([x for x in gt_terms if x != ''])+1e-8)
    f1 = 2 * p * r / (p + r) if p + r > 0 else 0.
    return [p, r, f1]


In [2]:
from evaluate import load
bertscore = load("bertscore")

In [3]:
P_list, R_list, F_list = [], [], []
for k, data in result.items():
    pred_list = data['pred']
    label_list = data['label']
    
    if len(label_list) > len(pred_list):
        label_list = label_list[:len(pred_list)]
    else:
        pred_list = pred_list[:len(label_list)]   
        
    results = bertscore.compute(predictions=pred_list, references=label_list, lang="en", device="cuda:0")
    precision, recall, f1 = results['precision'], results['recall'], results['f1']
    P, R, F = sum(precision)/len(precision), sum(recall)/len(recall), sum(f1)/len(f1)    
    P_list.append(precision)
    R_list.append(recall)
    F_list.append(f1)
    
    if (0 in precision) or (0 in recall) or (0 in f1):
        print(pred_list)
        print(label_list)
        break        

['are medicare services taxable in california', 'are medicare', 'social security', '']
['design services', 'consulting services', 'insurance services', 'construction services']


In [4]:
pred_list

['are medicare services taxable in california',
 'are medicare',
 'social security',
 '']

In [25]:
predictions = batch_pred[0]
references = batch_label[0][:2]
results = bertscore.compute(predictions=predictions, references=references, lang="en", device="cuda:0")

In [27]:
results = bertscore.compute(predictions=predictions, references=references, lang="en", device="cuda:0")

In [30]:
precision, recall, f1 = results['precision'], results['recall'], results['f1']
precision, recall, f1

([0.958801805973053, 0.9332197904586792],
 [0.9588017463684082, 0.9316922426223755],
 [0.958801805973053, 0.9324553608894348])

In [ ]:
groundtruth = ["for sale", "used cars", "electric", "cheap"]
cand = ["afforable cars", "cars for sale", "used", "electric"]
cand = best_bleu_cand(groundtruth, cand)

In [4]:
term_overlap_metrics = term_match(groundtruth, cand)
print("Term overlap metrics: P={},R={},F1={}".format(term_overlap_metrics[0],
                                                     term_overlap_metrics[1],
                                                     term_overlap_metrics[2]))

exact_match_metrics = exact_match(groundtruth, cand)
print("Exact match metrics: P={},R={},F1={}".format(exact_match_metrics[0],
                                                    exact_match_metrics[1],
                                                    exact_match_metrics[2]))

Term overlap metrics: P=0.8333333319444445,R=0.8333333319444445,F1=0.8333333319444444
Exact match metrics: P=0.249999999375,R=0.249999999375,F1=0.249999999375


In [5]:
eval_bleu(groundtruth, cand)

/home/jovyan/venv_p5/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jovyan/venv_p5/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jovyan/venv_p5/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF

[0.52904378163263,
 0.47141237159386706,
 0.4675886902868809,
 0.46146832211337435]

In [5]:
bert_score_metrics = bertscore(groundtruth, cand)
print("BERT score metrics: P={},R={},F1={}".format(bert_score_metrics[0],
                                                   bert_score_metrics[1],
                                                   bert_score_metrics[2]))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT score metrics: P=0.9065482020378113,R=0.9197155237197876,F1=0.9125113487243652
